In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
import  matplotlib.pyplot as plt
import cv2
import PIL
from tqdm import tqdm
import shutil

In [ ]:
data_path = "/kaggle/input/dataset1/agument"
content = os.listdir(data_path)

In [ ]:
content

In [ ]:
img_path = "/kaggle/input/dataset1/agument"


In [ ]:
counts = {}

for l in content:
    counts[l] = len(os.listdir(os.path.join(img_path, l)))

    
plt.figure(figsize=(12, 6))

plt.bar(range(len(counts)), list(counts.values()), align='center')
plt.xticks(range(len(counts)), list(counts.keys()), fontsize=12, rotation=40)
plt.xlabel('class label', fontsize=13)
plt.ylabel('class size', fontsize=13)
plt.title('Cyst , stone , normal, tumors', fontsize=15);

In [ ]:
'''img_paths1 = [os.path.join(img_path +"/"+l+"/"+l+'- (10).jpg') for l in content]

img_paths = img_paths1 + [os.path.join(img_path+"/"+l+"/"+l+'- (100).jpg') for l in content]

def plot_sat_imgs(paths):
    plt.figure(figsize=(15, 8))
    for i in range(8):
        plt.subplot(4, 5, i+1, xticks=[], yticks=[])
        img = PIL.Image.open(paths[i], 'r')
        plt.imshow(np.asarray(img))
        plt.title(paths[i].split('/')[-2])

plot_sat_imgs(img_paths)'''

In [ ]:
batch_size = 32
input_shape = (299,299,3)
channels = 3

In [ ]:
#image generator in tensorflow
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
os.mkdir('/kaggle/working/train_dir')
os.mkdir('/kaggle/working/test_dir')

In [ ]:
os.mkdir('/kaggle/working/Augument_dir')

In [ ]:
import os
from PIL import Image

# Specify the input and output directories
input_dir = "/kaggle/input/dataset1/agument"
output_dir = "/kaggle/working/Augument_dir"

# Loop through all files in the input directory
for filename in os.listdir(input_dir):
        for i in os.listdir(input_dir+"/"+filename):
            if not os.path.exists(os.path.join(output_dir,filename)):
                os.mkdir(os.path.join(output_dir,filename))
                image = Image.open(os.path.join(input_dir,filename, i))
                # Convert the image to PNG format
                output_filename = i+ ".jpeg"
                image.save(os.path.join(output_dir,filename, output_filename), "JPEG")
            else:
                image = Image.open(os.path.join(input_dir,filename, i))
                output_filename = i + ".jpeg"
                image.save(os.path.join(output_dir,filename, output_filename), "JPEG")


In [ ]:
content = os.listdir(input_dir)
for i in content:
    print(len(os.listdir(os.path.join(output_dir,i))))

In [ ]:
import re
from sklearn.model_selection import StratifiedShuffleSplit

TRAIN_DIR = '/kaggle/working/train_dir'
TEST_DIR = '/kaggle/working/test_dir'
BATCH_SIZE = batch_size
NUM_CLASSES=len(content)
INPUT_SHAPE = (299, 299, 3)
CLASS_MODE = 'categorical'
for path in (TRAIN_DIR, TEST_DIR):
    if not os.path.exists(path):
        os.mkdir(path)
for l in content:
    
    if not os.path.exists(os.path.join(TRAIN_DIR, l)):
        os.mkdir(os.path.join(TRAIN_DIR, l))

    if not os.path.exists(os.path.join(TEST_DIR, l)):
        os.mkdir(os.path.join(TEST_DIR, l))

In [ ]:
img_path = "/kaggle/working/Augument_dir"
data ={}
for l in content:
    for img in os.listdir(img_path+'/'+l):
        data.update({os.path.join(img_path+'/'+l+'/'+ img): l})

In [ ]:
#creating an image data frame
x = pd.Series(list(data.keys()))
y = pd.get_dummies(pd.Series(data.values()))
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=69)

In [ ]:
img = PIL.Image.open(x[0], 'r')
plt.imshow(np.asarray(img))

In [ ]:
for train_idx, test_idx in split.split(x, y):
    
    train_paths = x[train_idx]
    test_paths = x[test_idx]
    new_train_paths = [re.sub('/kaggle/working/Augument_dir', '/kaggle/working/train_dir', i) for i in train_paths]
    new_test_paths = [re.sub('/kaggle/working/Augument_dir', '/kaggle/working/test_dir', i) for i in test_paths]

    train_path_map = list((zip(train_paths, new_train_paths)))
    test_path_map = list((zip(test_paths, new_test_paths)))
    
    print("moving training files..")
    for i in tqdm(train_path_map):
        if not os.path.exists(i[1]):
            if not os.path.exists(re.sub('training', 'testing', i[1])):
                shutil.copy(i[0], i[1])
    
    print("moving testing files..")
    for i in tqdm(test_path_map):
        if not os.path.exists(i[1]):
            if not os.path.exists(re.sub('training', 'testing', i[1])):
                shutil.copy(i[0], i[1])

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
train_dir ='/kaggle/working/train_dir'
test_dir ='/kaggle/working/test_dir'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,# Rescaling the images
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    brightness_range=None,
    cval=0.0,
    vertical_flip=True,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    interpolation_order=1,
    dtype=None
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),  # Resize all images to 150x150
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
print(train_generator.class_indices)

In [ ]:
N_STEPS = train_generator.samples//BATCH_SIZE
N_VAL_STEPS = test_generator.samples//BATCH_SIZE
N_EPOCHS = 100

In [ ]:
#model building
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense , Flatten,Dropout,MaxPooling2D
from keras.optimizers import Adam , Adagrad,Adadelta

In [ ]:
from keras.applications import VGG16,VGG19
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, fbeta_score, accuracy_score

In [ ]:
conv_base = VGG16(include_top=False,
                              weights='/kaggle/input/model-weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 
                              input_shape=input_shape)
model = conv_base.output

model = Flatten()(model)
model = Dense(2048, activation='relu')(model)
model = Dropout(0.2)(model)
model = Dense(2048, activation='relu')(model)
model = Dropout(0.2)(model)
output_layer = Dense(len(content), activation='softmax')(model)
model = Model(inputs=conv_base.input, outputs=output_layer)
        

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def custom_vgg16(input_shape, num_classes):
    model = Sequential()

    # Convolutional layers
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Example usage:
input_shape = (299, 299, 3)  # Example input shape for image data
num_classes = 4  # Number of output classes
model = custom_vgg16(input_shape, num_classes)

# Print model summary
model.summary()


In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
 model.compile(optimizer='Adam', loss='categorical_crossentropy',
                 metrics=['categorical_accuracy'])

In [ ]:
model.summary()

In [ ]:

history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Number of batches per epoch
    epochs=50,
    validation_data=test_generator,
    validation_steps=50  # Number of batches for validation
)


In [ ]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
# model.compile(optimizer=optimizer, loss='mean_squared_error')

# # Train the model
# history = model.fit(train_data, train_labels, epochs=50, validation_data=(val_data, val_labels))


In [ ]:
model1 = tf.keras.applications.InceptionV3(
    include_top=True,
    weights='/kaggle/input/inceptions/inception_v3_weights_tf_dim_ordering_tf_kernels.h5',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np

base_model = InceptionV3(weights='/kaggle/input/inceptions/inception_v3_weights_tf_dim_ordering_tf_kernels.h5', include_top=True)
x = base_model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

In [ ]:
model1 = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


model1.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#model1.summary()

In [ ]:
model1.fit(
    train_generator,
    steps_per_epoch=100,  # Number of batches per epoch
    epochs=30,
    validation_data=test_generator,
    validation_steps=50
)

In [ ]:
model1.save('inceptionv3.h5')

In [ ]:
# Load and preprocess an example image
img_path = '/kaggle/working/train_dir/Cyst/Cyst- (687)_0_1764.jpeg.jpeg'  # Change this to the path of your image
img = load_img(img_path, target_size=(299, 299))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Predict the class of the image
preds = model.predict(img_array)
print('Predicted:', decode_predictions(preds, top=3)[0])

# Fine-tuning (optional)
# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model after unfreezing
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Now you can train your model on your dataset
# model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))